In [2]:
%load_ext autoreload
%autoreload 2

import os
import h5py
import pickle

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import illustris_python as il

%matplotlib inline
plt.style.use('/mnt/home/tnguyen/default.mplstyle')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
def get_mw_idx(cat, h=0.6909):
    """ Function to get MW index from the FoF catalog """

    masses = cat['GroupMassType'][:] * 1e10 / h
    tot_masses = np.sum(cat['GroupMassType'][:],axis=1) * 1e10 / h
    mcut = (tot_masses > 7e11) & (tot_masses < 3e12)
    if np.sum(mcut) == 0:
        return None
    cont = masses[:,2] / tot_masses
    idx = np.argmin(cont[mcut])
    if cont[mcut][idx] > .1:
        return None
    mw_idx = np.arange(len(masses))[mcut][idx]

    return mw_idx

def get_all_params():

    sample = np.loadtxt("/mnt/ceph/users/jrose/MW_zooms/fixed_cosmo/sobol_params.txt")

    Min = np.array([1/30, 0.25, 0.5, 0.25])
    Max = np.array([1/1.8, 4.0, 2.0, 4.0])

    Min[1:] = np.log10(Min[1:])
    Max[1:] = np.log10(Max[1:])

    params = Min + sample*(Max-Min)
    params[:,1:] = np.power(10, params[:,1:])
    params *= np.array([1, 3.6, 7.4, .1])
    params[:,0] = 1/params[:,0]

    return params

In [58]:
root_dir = "/mnt/ceph/users/jrose/MW_zooms/fixed_cosmo"
save_dir = "/mnt/ceph/users/tnguyen/dark_camels/point-cloud-diffusion-datasets/fof_h5_datasets"
save_name = "mw_zooms-wdm.pkl"
snap_num = 90
scale_factor = 1.0
sobol_params = get_all_params()

os.makedirs(save_dir, exist_ok=True)

In [79]:
halo_props = {
    'box_num': [],
    'pos': [],
    'vel': [],
    'mass': [],
    'mass_type': [],
    'subhalo_count': [],
    'subhalo_pos': [],
    'subhalo_vel': [],
    'subhalo_mass': [],
    'subhalo_mass_type': [],
    'sobol_params': []
}

for box_num in range(2000):
    fof_name = "box_{:d}/zoom/RUNs/output/fof_subhalo_tab_{:03d}.hdf5".format(
        box_num, snap_num)
    fof_path = os.path.join(root_dir, fof_name)

    # print("Reading box {:d}...".format(box_num))
    if not os.path.exists(fof_path):
        print(f"Box {box_num} does not exist. Skipping...")
        continue

    with h5py.File(fof_path, 'r') as f:
        mw_idx = get_fof_idx(f['Group'], f['Parameters'].attrs['HubbleParam'])
        if mw_idx is None:
            print("MW not found in box {:d}...".format(box_num))
            continue

        # read the positions, velocities and masses of the MW halo
        pos_halo = f['Group/GroupPos'][mw_idx] / 1e3       # positions in Mpc/h
        vel_halo = f['Group/GroupVel'][mw_idx] / scale_factor  # velocities in km/s
        mass_halo = f['Group/GroupMass'][mw_idx] * 1e10         # masses in Msun/h
        mass_type_halo = f['Group/GroupMassType'][mw_idx] * 1e10 # mass of each particle type in Msun/h

        # read the positions, velocities and masses of the subhalos of the MW
        group_number = f['Subhalo/SubhaloGrNr'][:]
        select = (group_number == mw_idx)
        pos_subhalos = f['Subhalo/SubhaloPos'][select] / 1e3         # positions in Mpc/h
        vel_subhalos = f['Subhalo/SubhaloVel'][select] / scale_factor  # velocities in km/s
        mass_subhalos = f['Subhalo/SubhaloMass'][select] * 1e10       # masses in Msun/h
        mass_type_subhalos = f['Subhalo/SubhaloMassType'][select] * 1e10 # mass of each particle type in Msun/h

        halo_props['box_num'].append(box_num)
        halo_props['pos'].append(pos_halo)
        halo_props['vel'].append(vel_halo)
        halo_props['mass'].append(mass_halo)
        halo_props['mass_type'].append(mass_type_halo)
        halo_props['subhalo_count'].append(len(pos_subhalos))
        halo_props['subhalo_pos'].append(pos_subhalos)
        halo_props['subhalo_vel'].append(vel_subhalos)
        halo_props['subhalo_mass'].append(mass_subhalos)
        halo_props['subhalo_mass_type'].append(mass_type_subhalos)

        # add the Sobol parameters
        halo_props['sobol_params'].append(sobol_params[box_num])

for p in halo_props:
    halo_props[p] = np.array(halo_props[p])

Box 0 does not exist. Skipping...
MW not found in box 21...
Box 27 does not exist. Skipping...
MW not found in box 42...
Box 66 does not exist. Skipping...
MW not found in box 88...
Box 89 does not exist. Skipping...
Box 92 does not exist. Skipping...
Box 159 does not exist. Skipping...
MW not found in box 223...
Box 237 does not exist. Skipping...
Box 244 does not exist. Skipping...
Box 252 does not exist. Skipping...
Box 257 does not exist. Skipping...
Box 306 does not exist. Skipping...
Box 321 does not exist. Skipping...
Box 338 does not exist. Skipping...
MW not found in box 342...
MW not found in box 372...
Box 406 does not exist. Skipping...
Box 418 does not exist. Skipping...
Box 429 does not exist. Skipping...
Box 453 does not exist. Skipping...
Box 464 does not exist. Skipping...
Box 474 does not exist. Skipping...
Box 501 does not exist. Skipping...
Box 509 does not exist. Skipping...
Box 519 does not exist. Skipping...
Box 520 does not exist. Skipping...
Box 521 does not ex

/tmp/ipykernel_3518111/1900933220.py:60: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  halo_props[p] = np.array(halo_props[p])


In [80]:
# save the halo properties to a file
with open(os.path.join(save_dir, save_name), 'wb') as f:
    pickle.dump(halo_props, f)